- Libraries

In [1]:
import os
from Inverted_Index import InvertedIndex
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

- Initialization

In [2]:
stemmer = PorterStemmer()
stopwords = stopwords.words("english")
iv = InvertedIndex()

- Preprocess

In [3]:
def preprocess():
    documents = []
    doc_path = os.getcwd()
    docs_folder_path = os.path.join(doc_path, "collection", "docs")
    
    for doc_id in os.listdir(docs_folder_path):
        file_path = os.path.join(docs_folder_path, doc_id)
        with open(file_path, 'r', encoding='utf-8') as file:
            for word in file:
                word = word.strip()
                stem = stemmer.stem(word)
                if stem not in stopwords:
                    iv.add_word(stem, doc_id)
                    
    # iv.print()
preprocess()

vs_inv_index:  {'pseudomona': {'DF': 79, 'TF': {'00001': 5, '00006': 1, '00007': 3, '00008': 1, '00018': 2, '00061': 2, '00062': 5, '00079': 2, '00080': 2, '00081': 5, '00082': 4, '00148': 1, '00152': 1, '00159': 2, '00160': 1, '00161': 2, '00176': 2, '00177': 3, '00178': 2, '00188': 5, '00200': 3, '00250': 1, '00260': 2, '00265': 2, '00271': 1, '00282': 3, '00384': 2, '00394': 1, '00427': 3, '00447': 2, '00451': 1, '00458': 2, '00459': 2, '00460': 1, '00469': 2, '00479': 2, '00505': 1, '00550': 2, '00556': 3, '00589': 1, '00590': 4, '00591': 1, '00611': 1, '00668': 2, '00696': 1, '00706': 1, '00753': 3, '00778': 9, '00790': 1, '00800': 3, '00802': 2, '00842': 1, '00865': 1, '00884': 2, '00890': 3, '00905': 7, '00912': 3, '00913': 2, '00917': 1, '00922': 1, '00983': 1, '00987': 2, '00989': 1, '01000': 1, '01010': 2, '01042': 3, '01065': 5, '01071': 3, '01077': 2, '01086': 2, '01089': 2, '01090': 2, '01091': 3, '01112': 1, '01171': 1, '01180': 1, '01192': 1, '01203': 3, '01227': 2}}, 'a